# Pattern recognition CW2

In [142]:
from scipy.io import loadmat
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
import time
from sklearn.cluster import KMeans

### Import file with 6 main components:

In [143]:
""" 
camId : which camera was used to get the shot (1 or 2)
filelist: names of the images (with format x_label_camId_index.png)
labels: class of the image (which person's image is it?)
query_idx: indexes of test set
gallery_idx: indexes of test set used for kNN 
train_idx: indexes of training and validation set
"""   
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')

### Import file with the feature vectors of all images:

In [144]:
import json
with open('PR_data/feature_data.json', 'r') as f:
    features = json.load(f)
features_np = np.array(features) #list of features converted to an array 
features_np.shape

(14096, 2048)

In [145]:
np.unique(train_idxs['labels'])[-1]

1467

In [188]:
train_idxs['labels'].flatten()

array([   1,    1,    1, ..., 1467, 1467, 1467], dtype=uint16)

In [189]:
train_idxs['labels'].shape

(14096, 1)

In [194]:
print(14096 in train_idxs['gallery_idx'].flatten())
print(14096 in train_idxs['train_idx'].flatten())
print(14096 in train_idxs['query_idx'].flatten())

False
True
False


In [148]:
7368+5328

12696

In [149]:
train_idxs['query_idx'].shape

(1400, 1)

In [150]:
12696+1400

14096

In [154]:
train_idxs['filelist'].shape

(14096, 1)

## kNN Classification for query set on gallery set

In [312]:
def remove_indices(n_indices, query_index):
    query_label_and_camId = (train_idxs['labels'][query_index][0], train_idxs['camId'][query_index][0])
    list_of_gallery_label_and_camId = [(label[0], camId[0]) for label, camId in zip(train_idxs['labels'][n_indices], train_idxs['camId'][n_indices])] 
    final_list = list(filter(lambda a: a != query_label_and_camId, list_of_gallery_label_and_camId))
    return [tup[0] for tup in final_list]

def get_accuracy(k, N):
    list_of_truths = []
    start = time.time() #time tracking - start time of process
    neigh = NearestNeighbors(n_neighbors=N, n_jobs=-1)
    neigh.fit(gallery_images)   #new_gallery_labels)
    for index, query_index in enumerate(query_indices.tolist()):
        if (index % 100 == 0):
            print("Index: ", index, " & time taken: ", time.time() - start)
        N_distances, N_indices = neigh.kneighbors(query_images[index].reshape(1, 2048))
        forLoopStart = time.time()
        topN_gallery_indices = (gallery_indices[N_indices[0]]).tolist()
        reduced_topN_labels = remove_indices(topN_gallery_indices, query_index)
        if query_labels[index][0] in reduced_topN_labels[:k]:
            list_of_truths.append(True)
        else:
            list_of_truths.append(False)
    #over-all accuracy  
    acc = sum(list_of_truths)/len(list_of_truths)
    print(acc)
    return acc

In [313]:
gallery_indices = train_idxs['gallery_idx'].flatten() - 1
gallery_images = features_np[gallery_indices]
gallery_labels = train_idxs['labels'][gallery_indices]

query_indices = train_idxs['query_idx'].flatten() - 1
query_images = features_np[query_indices]
query_labels = train_idxs['labels'][query_indices]

training_indices = train_idxs['train_idx'].flatten() - 1
training_images = features_np[training_indices]
training_labels = train_idxs['labels'][training_indices]

In [314]:
N=100
k = [1, 5, 10]
for k_val in k: 
    print("For rank ", k_val, ": ")
    get_accuracy(k_val, N)

For rank  1 : 
Index:  0  & time taken:  0.6711568832397461
Index:  100  & time taken:  11.172960996627808
Index:  200  & time taken:  21.65892195701599
Index:  300  & time taken:  32.16593790054321
Index:  400  & time taken:  42.716115951538086
Index:  500  & time taken:  53.33190989494324
Index:  600  & time taken:  63.814366817474365
Index:  700  & time taken:  74.30272483825684
Index:  800  & time taken:  84.79265689849854
Index:  900  & time taken:  95.2760329246521
Index:  1000  & time taken:  105.76963686943054
Index:  1100  & time taken:  116.35101675987244
Index:  1200  & time taken:  126.84344291687012
Index:  1300  & time taken:  137.45032286643982
0.47
For rank  5 : 
Index:  0  & time taken:  0.6394927501678467
Index:  100  & time taken:  11.25248098373413
Index:  200  & time taken:  21.72057294845581
Index:  300  & time taken:  32.267863035202026
Index:  400  & time taken:  42.73586893081665
Index:  500  & time taken:  53.28752779960632
Index:  600  & time taken:  63.81535

In [ ]:
k = 1, Accuracy = 0.47
k = 5, Accuracy = 0.6685714285714286
k = 10, Accuracy = 0.7492857142857143

## K-means

In [330]:
total_number_of_labels = np.unique(train_idxs['labels'])[-1]
n_clusters = 3 #total_number_of_labels
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(gallery_images)
index_and_cluster = [(index, label) for index, label in zip(gallery_indices, kmeans.labels_)]

In [350]:
def img_indices_in_cluster(c): 
    gallery_indices_in_cluster = [pair[0] for pair in index_and_cluster if pair[1] == c]
    return gallery_indices_in_cluster

def img_cluster(k, N, c_g_indices, query_index, index):
    list_of_truths = []
    neigh = NearestNeighbors(n_neighbors=N, n_jobs=-1, metric='euclidean')
    neigh.fit(features_np[c_g_indices])   #new_gallery_labels)
    
    N_distances, N_indices = neigh.kneighbors(features_np[query_index].reshape(1, -1))
    topN_c_g_indices = gallery_indices[N_indices[0]].flatten().tolist()
    reduced_topN_labels = remove_indices(topN_c_g_indices, query_index)
    return (query_labels[index][0] in reduced_topN_labels[:k])

def knn_clustering(k, N, cluster):
    class_success = []
    start = time.time() #time tracking - start time of process
    for index, query_index in enumerate(query_indices.tolist()):
        print(query_index, query_labels[index])
        if (index % 200 == 0):
            print("Index: ", index, " & time taken: ", time.time() - start)
        cluster = kmeans.predict(query_images[index].reshape(1,-1))
        cluster_gallery_indices = img_indices_in_cluster(cluster)
        print(cluster_gallery_indices)
        class_success.append(img_cluster(k, N, cluster_gallery_indices, query_index, index))
        break
     #over-all accuracy
    acc = sum(class_success)/len(class_success)
    print(acc)
    return acc

In [351]:
k = [1, 5, 10]
N = 100
acc_kmeans = []
for k_val in k:
    acc_kmeans.append(knn_clustering(k_val, N, cluster))

[0]
21 [3]
Index:  0  & time taken:  0.00028514862060546875
[47, 50, 51, 52, 53, 55, 56, 97, 98, 100, 150, 151, 152, 154, 155, 157, 158, 159, 205, 206, 207, 208, 292, 293, 295, 296, 297, 298, 299, 301, 304, 313, 314, 315, 316, 319, 320, 321, 322, 323, 324, 327, 330, 351, 352, 353, 354, 357, 358, 359, 360, 374, 376, 381, 383, 384, 385, 386, 389, 390, 391, 392, 393, 396, 397, 398, 420, 422, 423, 424, 425, 426, 427, 428, 441, 442, 443, 444, 446, 447, 448, 449, 507, 508, 510, 511, 512, 513, 520, 522, 523, 544, 545, 546, 548, 549, 550, 551, 563, 564, 565, 568, 569, 570, 571, 583, 584, 585, 586, 587, 588, 589, 591, 640, 641, 642, 644, 645, 646, 647, 648, 651, 652, 653, 654, 655, 656, 657, 659, 677, 679, 680, 681, 682, 683, 684, 686, 697, 698, 700, 701, 702, 703, 705, 706, 757, 759, 760, 761, 763, 765, 767, 768, 770, 771, 772, 774, 775, 776, 845, 846, 847, 849, 850, 851, 852, 854, 866, 867, 868, 869, 870, 872, 873, 874, 912, 913, 914, 915, 917, 918, 919, 920, 952, 953, 960, 961, 962, 964, 965

0.0
[0]
21 [3]
Index:  0  & time taken:  0.0001671314239501953
[47, 50, 51, 52, 53, 55, 56, 97, 98, 100, 150, 151, 152, 154, 155, 157, 158, 159, 205, 206, 207, 208, 292, 293, 295, 296, 297, 298, 299, 301, 304, 313, 314, 315, 316, 319, 320, 321, 322, 323, 324, 327, 330, 351, 352, 353, 354, 357, 358, 359, 360, 374, 376, 381, 383, 384, 385, 386, 389, 390, 391, 392, 393, 396, 397, 398, 420, 422, 423, 424, 425, 426, 427, 428, 441, 442, 443, 444, 446, 447, 448, 449, 507, 508, 510, 511, 512, 513, 520, 522, 523, 544, 545, 546, 548, 549, 550, 551, 563, 564, 565, 568, 569, 570, 571, 583, 584, 585, 586, 587, 588, 589, 591, 640, 641, 642, 644, 645, 646, 647, 648, 651, 652, 653, 654, 655, 656, 657, 659, 677, 679, 680, 681, 682, 683, 684, 686, 697, 698, 700, 701, 702, 703, 705, 706, 757, 759, 760, 761, 763, 765, 767, 768, 770, 771, 772, 774, 775, 776, 845, 846, 847, 849, 850, 851, 852, 854, 866, 867, 868, 869, 870, 872, 873, 874, 912, 913, 914, 915, 917, 918, 919, 920, 952, 953, 960, 961, 962, 964, 

0.0
[0]
21 [3]
Index:  0  & time taken:  0.00016498565673828125
[47, 50, 51, 52, 53, 55, 56, 97, 98, 100, 150, 151, 152, 154, 155, 157, 158, 159, 205, 206, 207, 208, 292, 293, 295, 296, 297, 298, 299, 301, 304, 313, 314, 315, 316, 319, 320, 321, 322, 323, 324, 327, 330, 351, 352, 353, 354, 357, 358, 359, 360, 374, 376, 381, 383, 384, 385, 386, 389, 390, 391, 392, 393, 396, 397, 398, 420, 422, 423, 424, 425, 426, 427, 428, 441, 442, 443, 444, 446, 447, 448, 449, 507, 508, 510, 511, 512, 513, 520, 522, 523, 544, 545, 546, 548, 549, 550, 551, 563, 564, 565, 568, 569, 570, 571, 583, 584, 585, 586, 587, 588, 589, 591, 640, 641, 642, 644, 645, 646, 647, 648, 651, 652, 653, 654, 655, 656, 657, 659, 677, 679, 680, 681, 682, 683, 684, 686, 697, 698, 700, 701, 702, 703, 705, 706, 757, 759, 760, 761, 763, 765, 767, 768, 770, 771, 772, 774, 775, 776, 845, 846, 847, 849, 850, 851, 852, 854, 866, 867, 868, 869, 870, 872, 873, 874, 912, 913, 914, 915, 917, 918, 919, 920, 952, 953, 960, 961, 962, 964,

0.0


### Mahalanobi Distance

In [359]:
len(np.unique(gallery_labels.flatten()))

700

Getting distances from the average gallery vector:

In [39]:
#number of unique labels in gallery 
len(np.unique(gallery_labels))

977

In [63]:
S = np.cov(np.transpose(gallery_images))
A = np.linalg.inv(S)

In [64]:
A.shape

(2048, 2048)

In [113]:
gallery_images.shape

(5328, 2048)

In [69]:
mean_gallery_images = gallery_images.mean(axis=0)     # to take the mean of each column (colummn -> feature)
mean_gallery_images.shape

(2048,)

In [79]:
query_set_distances = [np.sqrt(np.dot(np.dot(np.transpose(query_image-mean_gallery_images),A), query_image-mean_gallery_images))
                      for query_image in query_images]
np.array(query_set_distances).shape

(1400,)

In [367]:
d = [(1, 3.342), (2, 32.43), (3, 1.63), (4, 1343), (5, 31)]


[(3, 1.63), (1, 3.342), (5, 31), (2, 32.43), (4, 1343)]

In [ ]:
Out[5]: [('B', 3), ('A', 2), ('A', 1), ('I', 1), ('J', 1)]

Getting distances using kNN and Mahalanobi Distance

In [92]:
neigh_md = NearestNeighbors(n_neighbors=N,metric='mahalanobis', 
            metric_params= {'V': np.cov(np.transpose(gallery_images))})
neigh_md.fit(gallery_images) 

NearestNeighbors(algorithm='auto', leaf_size=30, metric='mahalanobis',
         metric_params={'V': array([[ 0.09833, -0.00735, ..., -0.00826, -0.00833],
       [-0.00735,  0.20639, ..., -0.00024, -0.00667],
       ...,
       [-0.00826, -0.00024, ...,  0.22353,  0.06679],
       [-0.00833, -0.00667, ...,  0.06679,  0.10489]])},
         n_jobs=1, n_neighbors=100, p=2, radius=1.0)

In [117]:
def get_accuracy_mahalanobi(k, N):
    list_of_truths = []
    start = time.time() #time tracking - start time of process
    for index, query_index in enumerate(train_idxs['query_idx'].flatten().tolist()[:300]):
        if (index % 50 == 0):
            print("Index: ", index, " & time taken: ", time.time() - start)
        N_distances, N_indices = neigh_md.kneighbors(query_images[index].reshape(1, 2048), N)
        topN_gallery_images = gallery_images[N_indices[0]]
        forLoopStart = time.time()
        topN_gallery_indices = train_idxs['gallery_idx'][N_indices[0]].flatten().tolist()
        reduced_topN_indices, reduced_topN_labels = remove_indices(topN_gallery_indices, query_index)
        if query_labels[index][0] in reduced_topN_labels[:k]:
            list_of_truths.append(True)
        else:
            list_of_truths.append(False)
    #over-all accuracy  
    acc = sum(list_of_truths)/len(list_of_truths)
    print("Avg Accuracy is: ", acc)
    return acc

In [118]:
N = 100
k = [1, 10, 100]
acc_k2 = []
for k_val in k: 
    acc = get_accuracy_mahalanobi(k_val, N)
    acc_k2.append(acc)

Index:  0  & time taken:  8.082389831542969e-05
Index:  50  & time taken:  1194.6821658611298
Index:  100  & time taken:  2801.5326719284058
Index:  150  & time taken:  4013.9011237621307
Index:  200  & time taken:  5203.121114730835
Index:  250  & time taken:  6391.557485818863
Avg Accuracy is:  0.13666666666666666
Index:  0  & time taken:  7.224082946777344e-05
Index:  50  & time taken:  1187.0171492099762
Index:  100  & time taken:  2373.66286110878
Index:  150  & time taken:  3560.3639900684357
Index:  200  & time taken:  4746.637277126312
Index:  250  & time taken:  5932.8617470264435
Avg Accuracy is:  0.2833333333333333
Index:  0  & time taken:  9.083747863769531e-05
Index:  50  & time taken:  1186.3573276996613
Index:  100  & time taken:  2372.715752840042
Index:  150  & time taken:  3559.498749732971
Index:  200  & time taken:  4745.957722663879
Index:  250  & time taken:  5932.393654823303
Avg Accuracy is:  0.4533333333333333


In [112]:
acc_k

[0.09090909090909091, 0.18181818181818182, 0.36363636363636365]

In [119]:
acc_k2

[0.13666666666666666, 0.2833333333333333, 0.4533333333333333]

### Cosine Distance

In [120]:
N = 100
neigh_c = NearestNeighbors(n_neighbors=N,metric='cosine')
neigh_c.fit(gallery_images) 

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=100, p=2, radius=1.0)

In [126]:
def get_acc_gen(k, N, index, query_index):
    N_distances, N_indices = neigh_c.kneighbors(query_images[index].reshape(1, 2048), N)
    topN_gallery_images = gallery_images[N_indices[0]]
    forLoopStart = time.time()
    topN_gallery_indices = train_idxs['gallery_idx'][N_indices[0]].flatten().tolist()
    reduced_topN_indices, reduced_topN_labels = remove_indices(topN_gallery_indices, query_index)
    return (query_labels[index][0] in reduced_topN_labels[:k])

k = [1, 10, 100]
acc_k3 = []
for k_val in k:
    list_of_truths = [] 
    start = time.time() #time tracking - start time of process
    for index, query_index in enumerate(train_idxs['query_idx'].flatten().tolist()[:300]):
        if (index % 50 == 0):
            print("Index: ", index, " & time taken: ", time.time() - start)
        truth_false = get_acc_gen(k_val, N, index, query_index)
        list_of_truths.append(truth_false)
    acc = sum(list_of_truths)/len(list_of_truths)
    print("Avg Accuracy is: ", acc)
    acc_k3.append(acc)

Index:  0  & time taken:  7.700920104980469e-05
Index:  50  & time taken:  4.227119207382202
Index:  100  & time taken:  8.423174142837524
Index:  150  & time taken:  12.630351066589355
Index:  200  & time taken:  16.83267116546631
Index:  250  & time taken:  21.04805612564087
Avg Accuracy is:  0.49
Index:  0  & time taken:  5.1975250244140625e-05
Index:  50  & time taken:  4.1925787925720215
Index:  100  & time taken:  8.413705825805664
Index:  150  & time taken:  12.6040678024292
Index:  200  & time taken:  16.792085886001587
Index:  250  & time taken:  20.98653793334961
Avg Accuracy is:  0.74
Index:  0  & time taken:  5.3882598876953125e-05
Index:  50  & time taken:  4.198216199874878
Index:  100  & time taken:  8.467583894729614
Index:  150  & time taken:  12.67863917350769
Index:  200  & time taken:  16.884459972381592
Index:  250  & time taken:  21.093831062316895
Avg Accuracy is:  0.91
